In [1]:
from imutils import face_utils #for resizing
import numpy as np
import argparse
import imutils
import dlib
import cv2
import time
from scipy.spatial import distance as dist #euclidian distance
import pandas as pd
import csv
from pathlib import Path

In [2]:
def write_dict_to_csv(csv_file, csv_columns, dict_data):
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            #writer.writeheader()
            for key, value in dict_data.items():
                writer.writerow({'name': key, 'face_data': value})
    except IOError:
        print("I/O error", csv_file)
    return
def append_to_csv(csvfile, data):
    with open(csvfile, 'a') as f:
        writer = csv.writer(f)
        for key, value in data.items():
            writer.writerow([key,value])
    return
def cvt_to_array(data, split_with=''):
    if split_with == '':
        return np.array(list(map(float, data)))
    else:
        return np.array(list(map(float, data.split(split_with))))
csv_columns = ['name', 'face_data']
csv_file = 'all_face_data.csv'

In [3]:
face_data = []
labels = []
data = {}

In [4]:
detector = dlib.get_frontal_face_detector() # detect the faces in the image. How many faces are there
predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat') # predict the face landmarks such as mouth or eyes
#(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"] 
#(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
facerec = dlib.face_recognition_model_v1('./dlib_face_recognition_resnet_model_v1.dat') #pretrained model. 
#we send the data to this function and it returns a 128D vector that described the faces.

In [5]:
#capture the person and save as the 128D vector
# this part captures only once
cap = cv2.VideoCapture(0)
#while True:

face_number = 0
while face_number == 0:
    print('Please show your whole face to camera. When the face is detected, you will be asked for the name.')
    time.sleep(0.5)
    ret, image = cap.read()
    image = imutils.resize(image, width=500) #resizing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #it should convert to gray in onder to improve resultt.
    rects = detector(gray, 0) # detect how many faces in the image
    cv2.imshow('asd', gray)
    for (i, rect) in enumerate(rects): 
        # for every faces
        # determine the facial landmarks for the face region, then
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect) # predict the face landmarks in image. 

        face_descriptor = facerec.compute_face_descriptor(image, shape) # send the shape data to resnet model. it returns a 128D vector
        
        while face_descriptor == -1:
            print('Face not found.')
        else:
            face_data.append(face_descriptor) # save the face data to array
            shape = face_utils.shape_to_np(shape)
            for (x, y) in shape:
                cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
            name = raw_input('who is this')
            labels.append(name)
            data[labels[0]] = face_data[0]
            face_data=[]
            labels=[]
            my_file = Path("./" + csv_file)
            if my_file.is_file():
                append_to_csv(csv_file, data)
                print('File already exist, data is appended to file')
            else:
                write_dict_to_csv(csv_file, csv_columns, data)    
                print('File has been created and data saved to file.')
            face_number += 1
        #print(face_descriptor)
        '''
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(image, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(image, [rightEyeHull], -1, (0, 255, 0), 1)
        ear = (leftEAR + rightEAR) / 2.0
        ''' 
cv2.imshow("Saved face", image)
cv2.waitKey(0)
#key = cv2.waitKey(1) & 0xFF
#break
# if the `q` key was pressed, break from the loopfurka
#if key == ord("q"):
#    break
#time.sleep(0.5)
cap.release()
cv2.destroyAllWindows()

Please show your whole face to camera. When the face is detected, you will be asked for the name.
who is thisdr dre
File already exist, data is appended to file


In [6]:
# this part is actually real time working part. It calcuates the same data as the previous part.
# and it calculates euclidian distance for every faces. If any calculations is less than 0.55 then it means we found faces

cap = cv2.VideoCapture(0)
while True:
    ret, image = cap.read()
    image = imutils.resize(image, width=200)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        
        trying = np.array(facerec.compute_face_descriptor(image, shape))
        #distance_faces = dist.euclidean(face_data, trying)
    
        with open('./'+csv_file, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                if row == [] or row[1] == "face_data":
                    continue
                else:
                    #row[1] = np.array(list(map(float, row[1].split('\n'))))
                    row[1] = cvt_to_array(row[1], '\n')
                    trying = cvt_to_array(trying)
                    distance_faces = dist.euclidean(row[1], trying)
                    if distance_faces < 0.55:
                        content = row[0]
                        break
                    else:
                        content = "unknown"
                    
        cv2.putText(image,content, (10,40), cv2.FONT_HERSHEY_PLAIN, 1, 255)
        
        
        shape = face_utils.shape_to_np(shape)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
        #print(distance_faces)
        '''if distance_faces < 0.55:
            cv2.putText(image,"furkan", (10,20), cv2.FONT_HERSHEY_PLAIN, 1, 255)
        else:
            cv2.putText(image,"unknown", (10,20), cv2.FONT_HERSHEY_PLAIN, 1, 255)
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(image, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(image, [rightEyeHull], -1, (0, 255, 0), 1)
        ear = (leftEAR + rightEAR) / 2.0
        
        
        
        '''
    cv2.imshow("Frame", image)
    key = cv2.waitKey(1) & 0xFF
    #break
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    time.sleep(0.1)
cap.release()
cv2.destroyAllWindows()    

In [36]:
cap.release()
cv2.destroyAllWindows()